In [2]:
!pip install GPUtil
import scipy.io
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import torch.nn as nn
import numpy as np
from tqdm import tqdm
import random
from PIL import Image
import glob
import math
import os
from IPython.display import HTML
import struct
import kornia
from skimage import feature
from GPUtil import showUtilization as gpu_usage
from torch.nn.utils.weight_norm import weight_norm

In [3]:
DATA_FOLDER = "/kaggle/input/the-small-norb-dataset-v10/"

PREFIXES = {
    'train': 'smallnorb-5x46789x9x18x6x2x96x96-training-',
    'test': 'smallnorb-5x01235x9x18x6x2x96x96-testing-',
}

FILE_TYPES = ['info', 'cat', 'dat']

# helper function to read int from file
def read_int(f):
    num, = struct.unpack('i', f.read(4))
    return num


# From https://cs.nyu.edu/~ylclab/data/norb-v1.0-small/ 
# "The magic number encodes the element type of the matrix"
# Note: I only copied over the ones I needed for these files.
map_magic_number_to_data_type = {
    '1e3d4c55': np.uint8,
    '1e3d4c54': np.int32,
}

loaded_data = {}

for dataset, prefix in PREFIXES.items():
    for filetype in FILE_TYPES:
        filename = prefix + filetype + ".mat"
        print('Reading {}'.format(filename))
        
        file_loc = os.path.join(DATA_FOLDER, filename)
        with open( file_loc, 'rb') as f:
            # Read the magic_num, convert it to hexadecimal, and look up the data_type
            raw_magic_num = read_int(f)
            magic_num = format(raw_magic_num, '02x')
            data_type = map_magic_number_to_data_type[magic_num]
            print('dtype', data_type)

            # Read how many dimensions to expect
            ndim = read_int(f)
            
            # Read at least 3 ints, or however many ndim there are
            shape = [
                read_int(f)
                for i in range(max(ndim, 3))
            ]   
            # But in case ndims < 3, take at most n_dim elements
            shape = shape[:ndim]
            print('shape', shape)
    
            # Now load the actual data!
            loaded_data[(dataset, filetype)] = np.fromfile(
                f, 
                dtype=data_type, 
                count=np.prod(shape)
            ).reshape(shape)

In [4]:
x_raw = torch.Tensor(loaded_data[('train', 'dat')]) / 255.
y = torch.Tensor(loaded_data[('train', 'cat')])
info = torch.Tensor(loaded_data[('train', 'info')])
x = torch.zeros(5,5,972,2,96,96)
count = 0
instances = [4,6,7,8,9]

for lab in range(5):
    
    for ins in range(5):
        
        count=0
        
        for lig in range(6):
        
            for ele in range(9):
            
                for azi in range(18):
                    
                    x[lab,ins,count] = \
                        x_raw[(y==lab)&\
                        (info[:,0]==instances[ins])&\
                        (info[:,1]==ele)&\
                        (info[:,2]==azi*2)&\
                        (info[:,3]==lig)]
                    count += 1

In [5]:
x = x.view(-1, 1, 96, 96)
x = x[:,:,10:-10,10:-10][torch.randperm(x.shape[0])].cuda()

In [6]:
N_TEMP = 80
T_SIZE = 13
F_COMP = 5
TAU = 0.1

In [7]:
class StructSpars(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        
        self.expander = weight_norm(nn.Conv2d(1, N_TEMP, T_SIZE, bias=False))
        
        self.encoder = nn.Conv2d(N_TEMP, N_TEMP, F_COMP, padding=F_COMP//2, bias=False)
                
        self.inhibitor = nn.Conv2d(N_TEMP, N_TEMP, F_COMP, padding=F_COMP//2, bias=True)

        self.decoder = weight_norm(nn.ConvTranspose2d(N_TEMP, N_TEMP, T_SIZE, groups=N_TEMP, bias=False))
                
        self.blender =  nn.Conv2d(N_TEMP, N_TEMP, F_COMP, padding=F_COMP//2, bias=True)
        
        
    def forward(self, x):
        
        expanded = torch.relu(self.expander(x))
        
        code = torch.relu(self.encoder(expanded))
                
        inh_factors = self.inhibitor(expanded)
        
        inh_tiles = F.unfold(inh_factors, F_COMP*2, stride=F_COMP) 
                        
        inh_tiles = F.gumbel_softmax(inh_tiles, TAU, dim=1)
                
        inh_mask = F.fold(inh_tiles, inh_factors.shape[-2:], F_COMP*2, stride=F_COMP)
                
        del inh_tiles, inh_factors, expanded
        
        overflow = inh_mask > 1.
        
        normaliser = torch.ones(inh_mask.shape, device=x.device) * (~ overflow) + overflow * inh_mask.detach()
        
        inh_mask = inh_mask / normaliser
            
        sparse_code = code * inh_mask
        
        del code, normaliser, overflow
        
        raw_reco = torch.relu(self.decoder(sparse_code))
                                
        ble_factors = self.blender(raw_reco)
        
        ble_mask = F.gumbel_softmax(ble_factors, TAU, dim=1)
                
        sparse_reco = raw_reco * ble_mask
        
        reco = torch.sum(sparse_reco, dim=1, keepdim=True)
                
        return reco, sparse_code, sparse_reco
    
    
    def get_templates(self):
        
        return self.decoder.weight.data
    
    def get_encoders(self):
        
        return self.expander.weight.data

In [8]:
def log_losses(reco, img, sparse_code, sparse_reco):
    
    diff = (reco - img) **2
    
    mse_loss = torch.mean(diff)
    
    activations = sparse_code.mean(dim=(0,2,3))

    code_spars_loss = (activations / activations.mean()).std()
                        
    activations = sparse_reco.mean(dim=(0,2,3))
        
    reco_spars_loss = (activations / activations.mean()).std()
    
    return 10*mse_loss, 0.1*code_spars_loss, 0.1*reco_spars_loss

In [9]:
def get_log(size, std):
    
    distance = torch.arange(size) - size//2
    
    x = distance.expand(1,1,size,size)**2
    
    y = x.transpose(-1,-2)
    
    t = (x + y) / (2*std**2)
    
    LoG = -1/(math.pi*std**4) * (1-t) * torch.exp(-t)
    
    return LoG

In [10]:
SHARPNESS = 75

plt.figure(figsize=(20,10))
plt.subplot(1,2,1)
plt.imshow(x[1,0].detach().cpu(), cmap="gray")

log = get_log(5, 0.7).cuda()
x = F.conv2d(x, log)
x = torch.relu(x)
x = torch.sigmoid(x*SHARPNESS)

x = x - torch.min(x.reshape(x.shape[0], -1), dim=1)[0].view(-1,1,1,1)
x = x / torch.max(x.reshape(x.shape[0], -1), dim=1)[0].view(-1,1,1,1)

plt.subplot(1,2,2)
plt.imshow(x[1,0].detach().cpu(), cmap="gray")

plt.show()
x.min(), x.max(), x.mean()

In [11]:
split = (x.shape[0]//100)*80
xtrain = x[:split]
xval = x[split:]
xtrain.shape, xval.shape
del x

In [12]:
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             
    torch.cuda.empty_cache()
    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()                           

In [13]:
def propagation(model, sample, b):
    
    reco, inh_mask, ble_mask = model(sample[b*B_SIZE:(b+1)*B_SIZE])
    
    mse, cod, tem = log_losses(reco, sample[b*B_SIZE:(b+1)*B_SIZE], inh_mask, ble_mask)
    
    loss = mse + cod + tem
    
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

In [14]:
def evaluation(model, sampleval, b):
    
    reco, inh_mask, ble_mask = model(sampleval[b*B_SIZE:(b+1)*B_SIZE])
    
    mse, cod, tem = log_losses(reco, sampleval[b*B_SIZE:(b+1)*B_SIZE], inh_mask, ble_mask)
    
    return mse, cod, tem

In [15]:
model = StructSpars().cuda()
optimiser = torch.optim.Adam(model.parameters())

In [16]:
EPOCHS = 10
B_SIZE = 128
nval = xval.shape[0] // B_SIZE
n_b = xtrain.shape[0] // B_SIZE
losstracker = torch.zeros((nval, 3)).cuda()
temptracker = torch.zeros((EPOCHS, N_TEMP*2, 1, T_SIZE, T_SIZE))

for e in range(EPOCHS): 

    temptracker[e,:N_TEMP] = model.get_templates().detach().cpu()
    temptracker[e,N_TEMP:] = model.get_encoders().detach().cpu()
    
    xtrain = xtrain[torch.randperm(xtrain.shape[0])]
    progress_bar = tqdm(range(n_b), position=0, leave=True)
    
    with torch.no_grad():
        
        for b in range(nval):
            
            losstracker[b,0],losstracker[b,1],losstracker[b,2] = \
                                                        evaluation(model, xval, b)
            
    progress_bar.set_description(\
                                    "E:"+ str(e) +\
                                    " MSE:" + format(losstracker[:,0].mean(),".3f") +\
                                    " COD:" + format(losstracker[:,1].mean(),".3f") +\
                                    " TEM:" + format(losstracker[:,2].mean(),".3f") +\
                                    " TOT:" + format(losstracker.sum(1).mean(),".3f"))
            
    for b in progress_bar:
        
        propagation(model, xtrain, b) 

In [17]:
torch.save(temptracker, 'temps.pt')
torch.save(model.state_dict(), 'weights.pth')

In [19]:
S = 56
testreco, code, blend = model(xval[S:S+2])
testreco = testreco.detach().cpu()
code = code.detach().cpu()
blend = blend.detach().cpu()

plt.figure(figsize=(30,10))
plt.subplot(1,4,1)
plt.imshow(testreco[0,0], cmap="gray")
plt.subplot(1,4,2)
plt.imshow(testreco[1,0], cmap="gray")
plt.subplot(1,4,3)
plt.imshow(xval[S,0].detach().cpu(), cmap="gray")
plt.subplot(1,4,4)
plt.imshow(xval[S+1,0].detach().cpu(), cmap="gray")
plt.show()

In [20]:
templates = model.get_templates().detach().cpu()
encoders = model.get_encoders().detach().cpu()
ROWS = 8

plt.figure(figsize=(30,30))

for i in range(N_TEMP):
    
    img = templates[i].permute(1,2,0)
    plt.subplot(ROWS*2, N_TEMP//ROWS, i+1)
    plt.imshow(img, cmap="gray")
    
for i in range(N_TEMP):
    
    img = encoders[i].permute(1,2,0)
    plt.subplot(ROWS*2, N_TEMP//ROWS, N_TEMP+i+1)
    plt.imshow(img, cmap="gray")
    
plt.show()

In [21]:
print((code.mean((0,-1,-2))*10000).round().int())
print((blend.mean((0,-1,-2))*10000).round().int())

In [22]:
plt.figure(figsize=(30,10))
S1 = 3

for i in range(N_TEMP):
    
    img = code[0,i].detach().cpu()
    plt.subplot(8, N_TEMP/4, i+1)
    plt.imshow(img, cmap="gray")
    
for i in range(N_TEMP):
    
    img = blend[0,i].detach().cpu()
    plt.subplot(8, N_TEMP/4, N_TEMP+i+1)
    plt.imshow(img, cmap="gray")
    
plt.show()

In [23]:
plt.figure(figsize=(30,10))
plt.subplot(1,4,1)
plt.imshow(code[1].mean((0)).detach().cpu())
plt.subplot(1,4,2)
plt.imshow(code[1].mean((0)).detach().cpu())
plt.subplot(1,4,3)
plt.imshow(blend[1].mean((0)).detach().cpu())
plt.subplot(1,4,4)
plt.imshow(blend[1,6].detach().cpu())

In [24]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
# weight norm improved stability in training but too small problem